In [1]:
#Feed Indexer tests [316-000000-043]

#Author: P.P.A. Kotze
#Date: 1/9/2020
#Version: 
#0.1 Initial
#0.2 Update after feedback and correction from HN email dated 1/8/2020
#0.3 Rework scu_get and scu_put to simplify
#0.4 attempt more generic scu_put with either jason payload or simple params, remove payload from feedback function

In [2]:
#Import of Python available libraries
import time
import requests
import json

In [3]:
#Setting of specific constants for particular DUT and test particulatrs
#Constants
#SCU connection constants
#SKA-MPI
#  scu_ip = '10.96.64.10'

#SIM @ home
scu_ip = '192.168.0.111'

port = '8080'

#script debug
debug = True

#SCU interface defaults
#Default slew speeds in deg/s
az_speed=2
el_speed=1

#Band definitions for indexer command
#bands={'Band 1' : 1, 'Band 2' : 2, 'Band 3' : 3, 'Band 4' : 4, 'Band 5a' : 5, 'Band 5b' : 6, 'Band 5c' : 7}

indexer_sensor_list=[
"acu.actual_timestamp",
"acu.azimuth.state",
"acu.azimuth.p_set",
"acu.azimuth.p_shape",
"acu.azimuth.p_act",
"acu.elevation.state",
"acu.elevation.p_set",
"acu.elevation.p_shape",
"acu.elevation.p_act",
"acu.feed_indexer.state",
"acu.feed_indexer.p_set",
"acu.feed_indexer.p_shape",
"acu.feed_indexer.p_act",
"acu.general_management_and_controller.feed_indexer_pos"
]

In [106]:
#Direct SCU webapi functions based on urllib PUT/GET

def feedback(r):
    if debug == True:
        print('***Feedback:', r.request.url, r.request.body)
        print(r.reason, r.status_code)
        print("***Text returned:")
        print(r.text)
    elif r.status_code != 200:
        print('***Feedback:', r.request.url, r.request.body)
        print(r.reason, r.status_code)
        print("***Text returned:")
        print(r.text)
        #print(r.reason, r.status_code)
    #print()

def scu_get(device, params = {}):
    '''This is a generic GET command into http: scu port + folder 
    with params=payload'''
    URL = 'http://' + scu_ip + ':' + port + device
    r = requests.get(url = URL, params = params)
    feedback(r)
    return(r)

def scu_put(device, payload = {}, params = {}):
    '''This is a generic PUT command into http: scu port + folder 
    with json=payload'''
    URL = 'http://' + scu_ip + ':' + port + device
    r = requests.put(url = URL, json = payload, params = params)
    feedback(r)
    return(r)

def scu_delete(device, payload = {},params = {}):
    '''This is a generic DELETE command into http: scu port + folder 
    with params=payload'''
    URL = 'http://' + scu_ip + ':' + port + device
    r = requests.delete(url = URL, json = payload, params = params)
    feedback(r)
    return(r)

In [5]:
#SIMPLE PUTS
    
#command authority
def command_authority(action):
    #1 get #2 release
    print('command authority: ', action)
    authority={'Get': 1, 'Release': 2}
    scu_put('/devices/command', 
            {'path': 'acu.command_arbiter.authority',
            'params': {'action': authority[action]}})
    
#commands to DMC state - dish management controller
def reset_dmc():
    print('reset dmc')
    scu_put('/devices/command', 
            {'path': 'acu.dish_management_controller.reset'})

def activate_dmc():
    print('activate dmc')
    scu_put('/devices/command',
            {'path': 'acu.dish_management_controller.activate'})

def deactivate_dmc():
    print('deactivate dmc')
    scu_put('/devices/command', 
            {'path': 'acu.dish_management_controller.deactivate'})

def move_to_band(position):
    bands = {'Band 1': 1, 'Band 2': 2, 'Band 3': 3, 'Band 4': 4, 'Band 5a': 5, 'Band 5b': 6, 'Band 5c': 7}
    print('move to band:', position)
    scu_put('/devices/command',
            {'path': 'acu.dish_management_controller.move_to_band',
             'params': {'action': bands[position]}})

def abs_azel(az_angle, el_angle):
    print('abs az: {:.3f} el: {:.3f}'.format(az_angle, el_angle))
    scu_put('/devices/command',
            {'path': 'acu.dish_management_controller.slew_to_abs_pos',
             'params': {'new_azimuth_absolute_position_set_point': az_angle,
                       'new_elevation_absolute_position_set_point': el_angle}})
    
#commands to ACU
def act_azimuth():
    print('act azimuth')
    scu_put('/devices/command', 
            {'path': 'acu.elevation.activate'})

def act_elevation():
    print('act elevation')
    scu_put('/devices/command', 
            {'path': 'acu.elevation.activate'})

def abs_azimuth(az_angle, az_vel):
    print('abs az: {:.3f} vel: {:.3f}'.format(az_angle, az_vel))
    scu_put('/devices/command',
            {'path': 'acu.azimuth.slew_to_abs_pos',
             'params': {'new_axis_absolute_position_set_point': az_angle,
              'new_axis_speed_set_point_for_this_move': az_vel}})    
    
def abs_elevation(el_angle, el_vel):
    print('abs el: {:.3f} vel: {:.3f}'.format(el_angle, el_vel))
    scu_put('/devices/command',
            {'path': 'acu.elevation.slew_to_abs_pos',
             'params': {'new_axis_absolute_position_set_point': el_angle,
              'new_axis_speed_set_point_for_this_move': el_vel}}) 


In [6]:
#SIMPLE GETS
def status_Value(sensor):
    r=scu_get('/devices/statusValue', 
              {'path': sensor})
    data = r.json()['value']
    #print('value: ', data)
    return(data)

def status_finalValue(sensor):
    #print('get status finalValue: ', sensor)
    r=scu_get('/devices/statusValue', 
              {'path': sensor})
    data = r.json()['finalValue']
    #print('finalValue: ', data)
    return(data)

def commandMessageFields(commandPath):
    r=scu_get('/devices/commandMessageFields', 
              {'path': commandPath})
    return(r)

def statusMessageField(statusPath):
    r=scu_get('/devices/statusMessageFields', 
              {'deviceName': statusPath})
    return(r)


In [197]:
#logger functions goes here

def create_logger(config_name, sensor_list):
    print('create logger')
    r=scu_put('/datalogging/config', 
              {'name': config_name,
               'paths': sensor_list})
    return(r)
    
'''unusual does not take json but params'''
def start_logger(filename):
    print('start logger: ', filename)
    r=scu_put('/datalogging/start',
              params='configName=' + filename)
    return(r)

def stop_logger():
    print('stop logger')
    r=scu_put('/datalogging/stop')
    return(r)

def logger_state():
    print('logger state ')
    r=scu_get('/datalogging/currentState')
    #print(r.json()['state'])
    return(r.json()['state'])

def logger_configs():
    print('logger configs ')
    r=scu_get('/datalogging/configs')
    return(r)

def logger_sessions():
    '''
    GET all sessions
    '''
    print('logger sessions ')
    r=scu_get('/datalogging/sessions')
    return(r)

def session_query(id):
    '''
    GET specific session only - specified by id number
    Usage:
    session_query('16')
    '''
    print('logger sessioN query id ')
    r=scu_get('/datalogging/session',
             {'id': id})
    return(r)

def session_delete(id):
    '''
    DELETE specific session only - specified by id number
    Not working - returns response 500
    Usage:
    session_delete('16')
    '''
    print('delete session ')
    r=scu_delete('/datalogging/session',
             params= 'id='+id)
    return(r)

def session_rename(id, new_name):
    '''
    RENAME specific session only - specified by id number and new session name
    Not working
    Works in browser display only, reverts when browser refreshed!
    Usage:
    session_rename('16','koos')
    '''    
    print('rename session ')
    r=scu_put('/datalogging/session',
             params = {'id': id, 
                        'name' : new_name})
    return(r)


def export_session(id, interval_ms=1000):
    '''
    EXPORT specific session - by id and with interval
    output r.text could be directed to be saved to file 
    Usage: 
    export_session('16',1000)
    or export_session('16',1000).text 
    '''
    print('export session ')
    r=scu_get('/datalogging/exportSession',
             params = {'id': id, 
                        'interval_ms' : interval_ms})
    return(r)

#sorted_sessions not working yet

def sorted_sessions(isDescending = 'True', startValue = '1', endValue = '25', sortBy = 'Name', filterType='indexSpan'):
    print('sorted sessions')
    r=scu_get('/datalogging/sortedSessions',
             {'isDescending': isDescending,
              'startValue': startValue,
              'endValue': endValue,
              'filterType': filterType, #STRING - indexSpan|timeSpan,
              'sortBy': sortBy})
    return(r)

#get latest session
def save_session(filename, interval_ms=1000, session = 'last',):
    '''
    Save session data as CSV after EXPORTing it
    Default interval is 1s
    Default is last recorded session
    if specified no error checking to see if it exists
    Usage: 
    export_session('16',1000)
    or export_session('16',1000).text 
    '''
    from pathlib import Path
    print('Attempt export and save of session: {} at rate {:.0f} ms'.format(session, interval_ms))
    if session == 'last':
        #get all logger sessions, may be many
        r=logger_sessions()
        #[-1] for end of list, and ['uuid'] to get id of last session in list
        session = (r.json()[-1]['uuid'])
    file_txt = export_session(session, interval_ms).text
    print('Session id: {} '.format(session))
    file_time = str(int(time.time()))
    file_name = str(filename + '-' + file_time + '.csv')
    file_path = Path.cwd()  / 'output' / file_name
    print('Log file location:', file_path)    
    f = open(file_path, 'a+')
    f.write(file_txt)
    f.close()





In [8]:
#Simplified one line commands particular to test section being peformed 

#wait seconds, wait value, wait finalValue
def wait_duration(seconds):
    print('wait for {:.1f}s'.format(seconds), end="")
    time.sleep(seconds)
    print(' done *')
    
def wait_value(sensor, value):
    print('wait until sensor: {} == value {}'.format(sensor, value))
    while status_Value(sensor) != value:
        time.sleep(1)
    print(' done *')

def wait_finalValue(sensor, value):
    print('wait until sensor: {} == value {}'.format(sensor, value))
    while status_finalValue(sensor) != value:
        time.sleep(1)
    print(' {} done *'.format(value))

In [ ]:
#Feed Indexer tests [316-000000-043] 

#Revisions:
#0.1 initial rewrite
#0.2 loop through all elevations, sugest change lower el from 15 to 20, move az and el independantly

#Common setup
config_name = 'IDX_LOG_6_8_8'
#log_filename = 'index_logger_' + str(int(time.time()))

loop1 = 10
loop2 = 5

#elevations = [15, 45, 85] lower limit not accessible

elevations = [20, 45, 85]
short_wait = 40
long_wait = 100


#test only remove these
'''
short_wait=1
long_wait=1
loop1=3
loop2=3
debug=True
'''
#Feed Indexer: Response & Repeatability (6.8.8)
'''
    a. Feed Indexer: Response & Repeatability (6.8.8)
    1) Manually move the Dish to 15° Elevation, any Azimuth angle (suitable for the current site conditions) 
       and FI at SPF Band5c and de-activate all axes.
    2) Manually start the LDS data recording
    3) Command the FI to the SPF B1 position,
        ◦  wait for 40 seconds to complete the move
    4) Command the FI to the SPF B2 position, 
        ◦ wait for 40 seconds to complete the move
    5) Command the FI to the SPF Band 5c position,
        ◦ wait for 40 seconds to complete the move
    6) Start the ACU data recording on the SCU using logging configuration TBD,
        ◦ wait for 60 seconds to stabilise
    7) Command the FI to SPF B1 position, 
        ◦ wait 40 seconds to complete the move
    8) Command the FI to SPF B5c position, 
        ◦ wait 100 seconds to complete the move and to stabilise
    9) Command the FI to SPF B2 position, 
        ◦ wait 40 seconds to complete the move
    10) Command the FI to SPF B5c position, 
        ◦ wait 100 seconds to complete the move and to stabilise
    11) Repeat steps 7) to 10) for a total of 10 cycles
    12) Command the FI to SPF B1 position, 
        ◦ wait 40 seconds to complete the move
    13) Command the FI to SPF B2 position, 
        ◦ wait 40 seconds to complete the move
    14) Repeat steps 12) to 13)  for a total of 5 cycles
    15) Command the FI to SPF B5c position, 
        ◦ wait 40 seconds to complete the move
    16) Stop the ACU data recording on the SCU
    17) Manually stop the LDS data recordings
    18) Extract the data on the SCU @ 5Hz sampling
    19) Rename the file to TBD (eg. 6p8p8_i.csv)
    20) Repeat the sequence 1) to 19) at 45° Elevation.
    21) Repeat the sequence 1) to 19) at 85° Elevation.
(Total duration ~ 1h per elevation angle => 3h)
'''
#Step 0
#Get command authority if available
ca_state = status_Value("acu.command_arbiter.act_authority")

if ca_state == '0':
    command_authority('Get')
    wait_value("acu.command_arbiter.act_authority", '3')
    
print(status_finalValue("acu.command_arbiter.act_authority"))

log_state=logger_state()
print(log_state)
if logger_state() == 'STOPPED':
    start_logger(config_name)
    wait_duration(short_wait)


for el_angle in elevations:
    
    #Step 1
    activate_dmc()
    wait_duration(short_wait)
    az_angle=0
    print('Select elevation angle: {:.3f}  \n'.format(el_angle))
    abs_azimuth(0,2)
    wait_duration(short_wait)
    abs_elevation(el_angle,1)
    wait_duration(short_wait)

    band = 'Band 5c'
    move_to_band(band)
    #wait_finalValue("acu.general_management_and_controller.feed_indexer_pos",band)
    wait_duration(short_wait)
    
    deactivate_dmc()

    #Step 2
    print('Start LDS recording here:')
    #input()

    #Step 3
    band='Band 1'
    move_to_band(band)
    wait_duration(short_wait)

    #Step 4
    band='Band 2'
    move_to_band(band)
    wait_duration(short_wait)

    #Step 5
    band='Band 5c'
    move_to_band(band)
    wait_duration(short_wait)

    #Step 6
    print('Start ACU recording? Already done at Step 0')

    #Loop Step 7 to 10, 10 times
    i=0
    while i < loop1:
        i+=1
        print('Loop 1 - Step 7 to 10: {:.0f} of {:.0f}'.format(i, loop1))
        
        #Step 7
        band='Band 1'
        move_to_band(band)
        wait_duration(short_wait)

        #Step 8
        band='Band 5c'
        move_to_band(band)
        wait_duration(long_wait)

        #Step 9
        band='Band 2'
        move_to_band(band)
        wait_duration(short_wait)

        #Step 10
        band='Band 5c'
        move_to_band(band)
        wait_duration(long_wait)
    #Step 11 loop end

    #Loop Step 12 to 13, 5 times
    i=0
    while i < loop2:
        i += 1
        print('Loop 2 - Step 12 to 13: {:.0f} of {:.0f}'.format(i, loop2))

        #Step 12
        band = 'Band 1'
        move_to_band(band)
        wait_duration(short_wait)

        #Step 13
        band = 'Band 2'
        move_to_band(band)
        wait_duration(short_wait)
    #Step 14 loop end

    #Step 15
    band='Band 5c'
    move_to_band(band)
    wait_duration(short_wait)

    #Step 16

    #End for elevation loop!
    print('End elevation: {:.3f}'.format(el_angle))
stop_logger()  
#Step 17
print('Stop LDS recording now :', end='')
#input()

#Step 18
print('Extract the data on the SCU @ 5Hz sampling')

#Step 19
print('Rename the file to TBD (eg. 6p8p8_i.csv)')

#Step 20
print('Repeat the sequence 1) to 19) at 45° Elevation.')

#Step 21
print('Repeat the sequence 1) to 19) at 85° Elevation.')

#Step 22


In [15]:
#Test setups
#config_name = 'IDX_LOG_6_10_2'
config_name = 'IDX_LOG_6_8_8'

#band_at_start = status_finalValue("acu.general_management_and_controller.feed_indexer_pos")
#bands = [ band_at_start, 'Band 1', 'Band 2']
bands_seq1 = [ 'Band 5c', 'Band 1', 'Band 2']
bands_seq2 = [ 'Band 1', 'Band 2', 'Band 5c']
bands_seq3 = [ 'Band 2', 'Band 5c', 'Band 1']
bands=bands_seq1

short_wait = 40
long_wait = 100
wait_80s = 80

el_start = 15
az_start = 0
elevations = [15, 25, 35, 45, 55, 65, 75, 85]

loop1 = 5

#debug logic

loop1 = 2
short_wait = 10
long_wait = 10
wait_80s = 15
debug = True

#b. Feed Indexer: Stability – gravity, increments, static (6.10.2-a1)
'''
    1) Manually move the Dish to 15° Elevation, any Azimuth angle (suitable for the current site conditions) 
    and FI at SPF Band5c and de-activate all axes
    2) Manually start the LDS (Lateral Displacement Sensor / Kinesis) data recording
    3) Activate the Elevation axis
        ◦ wait for 15 seconds to complete the activation process (note the current SPF Band position(1))
    4) Command the FI to the SPF B1 position,
        ◦  wait for 40 seconds to complete the move
    5) Command the FI to the SPF B2 position, 
        ◦ wait for 40 seconds to complete the move
    6) Command the FI to the initial position(1),
        ◦ wait for 40 seconds to complete the move
    7) Start the ACU data recording on the SCU using logging configuration TBD,
        ◦ wait for 60 seconds to stabilise
    8) Command elevation upwards in steps of 10° (15/25/35/45/55/65/75/85) at a maximum rate of 1°/s, 
        ◦ wait for 80 seconds after each step command
    9) Once at 85° repeat step 8) downwards to 15°
    10) Repeat steps 8) and 9) for a total of 5 cycles
    11) Stop the ACU data recording on the SCU
    12) De-activate the Elevation axis
    13) Manually stop the LDS data recording
    14) Extract the data on the SCU @ 5Hz sampling
    15) Rename the file to TBD (eg. 6p10p2a1_i.csv)
    16) Repeat the sequence 1) to 15) with the FI at the SPF Band 1 starting position
    17) Repeat the sequence 1) to 15) with the FI at the SPF Band 2 starting position

(Total duration ~ 1h45min per SPF Band => 5h30min)

(1) The script to sample the current Elevation angle position at the start of the script and use it 
as the “applicable SPF Band” since the test will be repeated for different SPF Band positions.
'''
#Step 0
log_state=logger_state()
print('Current logger state is: {}',format(log_state))
if log_state == 'STOPPED':
    start_logger(config_name)
    wait_duration(short_wait)
elif log_state == 'RECORDING':
    print('Logger already running')
    input()
          
print('Band sequence {}'.format(bands))    
activate_dmc()
if debug == True:
    wait_duration(60)
wait_duration(short_wait)

#Step 1
abs_azimuth(az_start, az_speed)
wait_duration(short_wait)
abs_elevation(el_start, el_speed)
wait_duration(short_wait)

band = bands[0]
move_to_band(band)
#wait_finalValue("acu.general_management_and_controller.feed_indexer_pos",band)
wait_duration(short_wait)
deactivate_dmc()
if debug == True:
    wait_duration(60)
    
#Step 2
print("Start LDS data recording")
#input()

#Step 3
activate_dmc()
if debug == True:
    wait_duration(60)
wait_duration(long_wait)

#Step 4
band=bands[1]
move_to_band(band)
wait_duration(short_wait)

#Step 5
band=bands[2]
move_to_band(band)
wait_duration(short_wait)

#Step 6
band = bands[0]
move_to_band(band)
wait_duration(short_wait) 

#Step 7
print('Start ACU recording here - but started at the start!?')

#Loop Step 8 and Step 9
i=0
while i < loop1:
    i+=1
    print('Loop 1 - Step 8 to 9: {:.0f} of {:.0f}'.format(i, loop1))

    #Step 8
    for el_angle in elevations[1:]:
        abs_elevation(el_angle, el_speed)
        wait_duration(wait_80s)
    #Step 9
    for el_angle in list(reversed(elevations))[1:]:
        abs_elevation(el_angle, el_speed)
        wait_duration(wait_80s)

    #Step 10 - end loop

#Step 11
print('Stop ACU recording here - well sorry about that')
stop_logger() 

#Step 12
deactivate_dmc()

#Step 13
print("Stop LDS data recording")

#Step 14
print("Extract the data on the SCU @ 5Hz sampling")

#Step 15
print("Rename the file to TBD (eg. 6p10p2a1_i.csv)")

#Step 16
# sequence 1) to 15) with the FI at the SPF Band 1 starting position
#Step 17
# the sequence 1) to 15) with the FI at the SPF Band 2 starting position


 

logger state 
***Feedback: http://192.168.0.111:8080/datalogging/currentState None
OK 200
***Text returned:
{"state":"STOPPED"}

Current logger state is: {} STOPPED
start logger:  IDX_LOG_6_8_8
***Feedback: http://192.168.0.111:8080/datalogging/start?configName=IDX_LOG_6_8_8 b'{}'
OK 200
***Text returned:


wait for 10.0s done *
Band sequence ['Band 5c', 'Band 1', 'Band 2']
activate dmc
***Feedback: http://192.168.0.111:8080/devices/command b'{"path": "acu.dish_management_controller.activate"}'
OK 200
***Text returned:
{"uuid":"f486b18b-03bf-4d57-98d7-6fa74b5c3367"}

wait for 60.0s done *
wait for 10.0s done *
abs az: 0.000 vel: 2.000
***Feedback: http://192.168.0.111:8080/devices/command b'{"path": "acu.azimuth.slew_to_abs_pos", "params": {"new_axis_absolute_position_set_point": 0, "new_axis_speed_set_point_for_this_move": 2}}'
OK 200
***Text returned:
{"uuid":"aea88e86-4a15-4945-b8a1-6f241de91fd1"}

wait for 10.0s done *
abs el: 15.000 vel: 1.000
***Feedback: http://192.168.0.111:808

KeyboardInterrupt: 

In [ ]:
#c. Feed Indexer: Stability – gravity, increments, cycling (6.10.2-a2)
'''
(Note this is similar to 6.10.2-a1 with additional FI motions)
    1) Manually move the Dish to 15° Elevation, any Azimuth angle (suitable for the current site conditions) 
    and FI at SPF Band5c and de-activate all axes
    2) Manually start the LDS data recording
    3) Activate the Elevation axis
        ◦ wait for 15 seconds to complete the activation process (note the current SPF Band position(1))
    4) Command the FI to the SPF B1 position,
        ◦  wait for 40 seconds to complete the move
    5) Command the FI to the SPF B2 position, 
        ◦ wait for 40 seconds to complete the move
    6) Command the FI to the initial position(1),
        ◦ wait for 40 seconds to complete the move
    7) Start the ACU data recording on the SCU using logging configuration TBD,
        ◦ wait for 60 seconds to stabilise
    8) Command elevation upwards in steps of 10° (15/25/35/45/55/65/75/85) at a maximum rate of 1°/s. With each step:
        ◦ wait for 80 seconds to complete the move and to stabilise
        ◦ command the FI to the SPF B1 position, 
            ▪ wait for 40 seconds to compete the move
        ◦ command the FI to the SPF B2 position, 
            ▪ wait for 40 seconds to complete the move
        ◦ command the FI to the initial position(1) 
            ▪ wait for 100 seconds to complete move and to stabilise
    9) Once at 85° repeat step 8) going downwards to 15°
    10) Repeat steps 8) and 9) for a total of 3 cycles
    11) Stop the ACU data recording on the SCU
    12) De-activate the Elevation axis
    13) Manually stop the LDS data recording
    14) Extract the data on the SCU @ 5Hz sampling
    15) Rename the file to TBD (eg. 6p10p2a2_i.csv)
    16) Repeat the sequence 1) to 15) with the FI at the SPF Band 1 position
    17) Repeat the sequence 1) to 15) with the FI at the SPF Band 2 position

(Total duration ~ 3h per SPF Band => ~9h)

(1) The script to sample the current SPF Band position at the start of the script and use it as the “applicable SPF Band” since the test will be repeated for different SPF Band positions.
 '''



In [14]:
print(logger_state())
stop_logger()

logger state 
***Feedback: http://192.168.0.111:8080/datalogging/currentState None
OK 200
***Text returned:
{"state":"RECORDING"}

RECORDING
stop logger
***Feedback: http://192.168.0.111:8080/datalogging/stop b'{}'
OK 200
***Text returned:
{"uuid":null,"startTime":"2020-09-04T06:45:34.736513100Z","stopTime":"2020-09-04T06:47:04.656934100Z","duration":"00:01:29","configId":23,"fullFilePath":".\\datalogging\\2020\\9\\04___06_45_34","name":"IDX_LOG_6_8_8"}



<Response [200]>

In [59]:
print(logger_state())
stop_logger()


logger state 
***Feedback: http://192.168.0.111:8080/datalogging/currentState None
OK 200
***Text returned:
{"state":"RECORDING"}

RECORDING
stop logger
***Feedback: http://192.168.0.111:8080/datalogging/stop b'{}'
OK 200
***Text returned:
{"uuid":null,"startTime":"2020-09-04T06:49:07.265906100Z","stopTime":"2020-09-04T07:34:29.778185400Z","duration":"00:45:22","configId":23,"fullFilePath":".\\datalogging\\2020\\9\\04___06_49_07","name":"IDX_LOG_6_8_8"}



<Response [200]>

In [181]:
print(logger_state())
#start_logger(config_name)
#stop_logger()

logger state 
STOPPED
